# Work in progress, does not work in mybinder.
Also due to travel restrictions you should run this at home on self isolated compute.

# Wash you hands after running

In [1]:
#!pwsh
Invoke-WebRequest -Uri "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv" -OutFile "./Confirmed.csv"
Invoke-WebRequest -Uri "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv" -OutFile "./Deaths.csv"
Invoke-WebRequest -Uri "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv" -OutFile "./Recovered.csv"

In [2]:
#r "nuget:Microsoft.Data.Analysis,0.2.0"

Installed package Microsoft.Data.Analysis version 0.2.0

In [3]:
using Microsoft.Data.Analysis;

var deaths = DataFrame.LoadCsv("./Deaths.csv");
var confirmed = DataFrame.LoadCsv("./Confirmed.csv");
var recovered = DataFrame.LoadCsv("./Recovered.csv");
var offset = 4;

var date = deaths.Columns[offset].Name;
var deathFiltered = deaths[deaths.Columns[offset].ElementwiseNotEquals(0)];
var confirmedFiltered = confirmed[confirmed.Columns[offset].ElementwiseNotEquals(0)];
var recoveredFiltered = recovered[recovered.Columns[offset].ElementwiseNotEquals(0)];

var deathsSeries = new {
    latitude = deathFiltered["Lat"],
    longitude = deathFiltered["Long"],
    data = deathFiltered.Columns[offset]
};
var confirmedSeries = new {
    latitude = confirmedFiltered["Lat"],
    longitude = confirmedFiltered["Long"],
    data = confirmedFiltered.Columns[offset]
};
var recoveredSeries = new {
    latitude = recoveredFiltered["Lat"],
    longitude = recoveredFiltered["Long"],
    data = recoveredFiltered.Columns[offset]
};

Unhandled exception: System.FormatException: Input string was not in a correct format.
   at System.Number.ThrowOverflowOrFormatException(ParsingStatus status, TypeCode type)
   at System.String.System.IConvertible.ToSingle(IFormatProvider provider)
   at System.Convert.ChangeType(Object value, Type conversionType, IFormatProvider provider)
   at Microsoft.Data.Analysis.DataFrame.Append(IEnumerable`1 row)
   at Microsoft.Data.Analysis.DataFrame.LoadCsv(Stream csvStream, Char separator, Boolean header, String[] columnNames, Type[] dataTypes, Int64 numberOfRowsToRead, Int32 guessRows, Boolean addIndexColumn)
   at Microsoft.Data.Analysis.DataFrame.LoadCsv(String filename, Char separator, Boolean header, String[] columnNames, Type[] dataTypes, Int32 numRows, Int32 guessRows, Boolean addIndexColumn)
   at Submission#4.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionStat

In [34]:
#!html
<div id="plotlyChartCovid"></div>
<script>
    plotlyJs_covid_require = require.config({
        context: "COVID",
        paths: {
            plotly: "https://cdn.plot.ly/plotly-latest.min"
        }
    });
    plotlyJs_covid_require(["plotly"], (Plotly) => {
         
        function updateCovidPlot(){
             Promise.all([
                 interactive.csharp.getVariable("deathsSeries"), 
                 interactive.csharp.getVariable("confirmedSeries"), 
                 interactive.csharp.getVariable("recoveredSeries"),
                 interactive.csharp.getVariable("date")])
            .then(data => {
                Plotly.animate("plotlyChartCovid", {
                    data:[
                        {
                            lat: data[2].latitude,
                    lon: data[2].longitude,
                    text: data[2].data
                        },
                        {lat: data[0].latitude,
                    lon: data[0].longitude,
                    text: data[0].data},
                        {
                            lat: data[1].latitude,
                    lon: data[1].longitude,
                    text: data[1].data
                        }],
                    layout:{
                        title: "COVID-19 "+ data[3]
                    }
                });
            });
        }
            Promise.all([
                interactive.csharp.getVariable("deathsSeries"), 
                interactive.csharp.getVariable("confirmedSeries"), 
                interactive.csharp.getVariable("recoveredSeries"),
                interactive.csharp.getVariable("date")])
            .then(data => {
                var recovered = {
                    name: "recovered",
                    type: "scattergeo",
                    mode: "markers",
                    geo: "geo1",
                    lat: data[2].latitude,
                    lon: data[2].longitude,
                    text: data[2].data,
                    marker: {
                        symbol: "square",
                        colorscale: "Viridis",
                    }
                };

                var deaths = {
                    name: "death",
                    type: "scattergeo",
                    geo: "geo2",
                    mode: "markers",
                    lat: data[0].latitude,
                    lon: data[0].longitude,
                    text: data[0].data,
                    marker: {
                        symbol: "circle",
                        colorscale: "Viridis",
                    }
                };

                var confirmed = {
                    name: "confirmed",
                    type: "scattergeo",
                    geo: "geo3",
                    mode: "markers",
                    lat: data[1].latitude,
                    lon: data[1].longitude,
                    text: data[1].data,
                    marker: {
                        symbol: "diamond",
                        colorscale: "Viridis",
                    }
                };

                var traces = [recovered, deaths, confirmed];

                var layout = {
                    title: "COVID-19 "+ data[3],
                    grid: { columns: 3, rows: 1 },
                    geo1: {
                        //resolution: 50,
                        scope: "world",
                        showland: true,
                        landcolor: "rgb(250,250,250)",
                        domain: {
                            row: 0,
                            column: 0
                        }
                    },
                    geo2: {
                        // resolution: 50,
                        scope: "world",
                        showland: true,
                        landcolor: "rgb(250,250,250)",
                        domain: {
                            row: 0,
                            column: 1
                        }
                    },
                    geo3: {
                        //  resolution: 50,
                        scope: "world",
                        showland: true,
                        landcolor: "rgb(250,250,250)",
                        domain: {
                            row: 0,
                            column: 2
                        }
                    }
                };

                Plotly.newPlot("plotlyChartCovid", traces, layout);
                setInterval(() => updateCovidPlot(),1000);
            });
    });
</script>

In [39]:
for(var i = offset; i <  deaths.Columns.Count; i++){
    await Task.Delay(1000);
    date = deaths.Columns[i].Name;
    deathFiltered = deaths[deaths.Columns[i].ElementwiseNotEquals(0)];
    confirmedFiltered = confirmed[confirmed.Columns[i].ElementwiseNotEquals(0)];
    recoveredFiltered = recovered[recovered.Columns[i].ElementwiseNotEquals(0)];

    deathsSeries = new {
        latitude = deathFiltered["Lat"],
        longitude = deathFiltered["Long"],
        data = deathFiltered.Columns[i]
    };
    confirmedSeries = new {
        latitude = confirmedFiltered["Lat"],
        longitude = confirmedFiltered["Long"],
        data = confirmedFiltered.Columns[i]
    };
    recoveredSeries = new {
        latitude = recoveredFiltered["Lat"],
        longitude = recoveredFiltered["Long"],
        data = recoveredFiltered.Columns[i]
    };
}